# Setup

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot

matplotlib.pyplot.ioff()

from package.apps import Apps

# Queries

## Apps in prod using JsonObjectInDataResponse

In [ ]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod"
        and app.app_settings.some(
            lambda app_settings: app_settings.environment in ["Production", "default"]
            and app_settings[".FeatureManagement.JsonObjectInDataResponse"] == True
        )
    ).table()

## Apps in prod using RemoveHiddenData

In [ ]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod"
        and app.app_settings.some(
            lambda app_settings: app_settings.environment in ["Production", "default"]
            and app_settings[".AppSettings.RemoveHiddenData"] == True
        )
    ).table()

## Apps in prod using RequiredValidation

In [ ]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod"
        and app.app_settings.some(
            lambda app_settings: app_settings.environment in ["Production", "default"]
            and app_settings[".AppSettings.RequiredValidation"] == True
        )
    ).table()

## Apps using old PDF generator

In [ ]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.backend_version.exists
        and app.backend_version.major < 8
        and not app.app_settings.some(lambda app_settings: app_settings[".FeatureManagement.NewPdfGeneration"] == True)
        and (
            app.backend_version < "1.1.11-alpha"
            or (
                app.backend_version >= "1.1.11-alpha"
                and app.backend_version < "4.33.0"
                and app.application_metadata[".dataTypes.[]?", :].some(lambda dataType: dataType[".appLogic"] != None)
            )
            or (
                app.backend_version >= "4.33.0"
                and app.backend_version < "5.3.0"
                and app.application_metadata[".dataTypes.[]?", :].some(
                    lambda dataType: dataType[".appLogic"] != None and dataType[".enablePdfCreation"] == True
                )
            )
            or (
                app.backend_version >= "5.3.0"
                and app.application_metadata[".dataTypes.[]?", :].some(
                    lambda dataType: dataType[".appLogic.classRef"] != None and dataType[".enablePdfCreation"] == True
                )
            )
        )
    ).select(
        {
            "Backend version": lambda app: app.backend_version,
            "Repo url": lambda app: app.repo_url,
            "App url": lambda app: app.app_url,
        }
    ).unique_repos().order_by(
        lambda app: (app.org, app.app)
    ).csv(
        "apps_using_old_pdf"
    ).table()